# The Price is Right

  ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

 20%|████████████████▊                                                                   | 1/5 [00:51<03:25, 51.37s/it]C:\Users\benja\projects\llm_engineering\week8\agents\deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:59<00:00, 47.99s/it]


In [4]:
len(deals)

47

In [5]:
deals[44].describe()

'Title: Scotts Turf Builder WinterGuard Fall Lawn Food 10-lb. Bag for $28 + free shipping w/ $35\nDetails: This best seller is $30 off and the best deal we could find. Opt for store pickup (where available) to dodge the $6.99 shipping fee or orders of $35 or more ships for free. Buy Now at Walmart\nFeatures: covers up to 4,000-sq. ft. Model: 22342\nURL: https://www.dealnews.com/products/Scotts/Scotts-Turf-Builder-Winter-Guard-Fall-Lawn-Food-10-lb-Bag/493656.html?iref=rss-c196'

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Apple iPhone 17 Pro & Pro Max (2025): Preorders starting September 12 + free shipping
Details: Get ready to order Apple's newest flagship iPhones on Friday, September 12. Both the 6.3" Pro and 6.9" Pro Max feature a brand new heat-forged aluminum unibody frame, 8x telephoto camera, faster charging, and improved battery life. Plus, you can trade in your old mobile fo

In [9]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [10]:
result = get_recommendations()

In [11]:
len(result.deals)

5

In [12]:
result.deals[1]

Deal(product_description="Introducing the Apple iPhone Air, the thinnest iPhone ever at just 5.6mm. It showcases a beautiful 6.5-inch Super Retina XDR display, powered by an A19 Pro chipset that guarantees performance and efficiency. The phone is crafted with a durable titanium frame, providing both elegance and sturdiness. The 48MP Fusion Camera system allows for high-quality imaging, while the device supports the latest wireless technologies, ensuring you'll stay connected effortlessly and enjoy all-day battery life.", price=799.0, url='https://www.dealnews.com/products/Apple/Apple-iPhone-Air-2025/493679.html?iref=rss-c142')

In [13]:
from agents.scanner_agent import ScannerAgent

In [14]:
agent = ScannerAgent()
result = agent.scan()

In [15]:
result

DealSelection(deals=[Deal(product_description='The Apple iPhone 17 Pro and Pro Max are the latest in Apple\'s flagship line, featuring a sleek 6.3" and 6.9" Pro display, respectively. Constructed with a cutting-edge heat-forged aluminum unibody frame, these models boast an impressive Pro Camera System that includes a 48MP Fusion Camera and an 8x telephoto lens. Equipped with the A19 Pro chipset, users can expect faster charging and improved battery life. Ideal for photography enthusiasts and tech aficionados alike, these devices are set to redefine mobile technology.', price=999.0, url='https://www.dealnews.com/products/Apple/Apple-iPhone-17-Pro-2025/493678.html?iref=rss-c142'), Deal(product_description='The Apple Watch Ultra 3 is the ultimate choice for fitness enthusiasts and tech-savvy individuals. With the largest display ever on an Apple Watch, this model features satellite connectivity for emergency SOS, messaging, and locating services. Battery life extends to an impressive 42 h